# Feature engineering
## Examining the data
First, we'll take a look at the data that we're working with

In [1]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


So we have 1460 samples, with 81 columns. The goal is to predict the last column, `SalePrice`.

## Processing the features for regression
This process will include:
1. Isolating the numerical features, setting any `NaN` cells to 0.
2. Isolating the categorical features, and encoding with `OneHotEncoder`.
3. Isolating ordinal categorical features, and encoding with `OrdinalEncoder`.

In [2]:
# first layer - split by datatype
num_df = df.select_dtypes(exclude='object')
cat_df = df.select_dtypes(include='object')
num_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [3]:
cat_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


### Processing numerical features
There are many features in the numerical dataset that are actually categorical by nature, like `YearBuilt`. We will manually examine and justify each feature we deem categorical in this set, and perform one-hot encoding.

In [4]:
num_df.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

- `Id`: this is a useless feature, as a house's internal identifying number doesn't determine its price -- this will later be removed.
- `MSSubClass`: this is an identifier for the type of dwelling involved in the sale. Since it's an identifier, it should be a categorical feature.
- `YearBuilt`: this identifies the year the house was built; it should be a categorical feature.
- `YearRemodAdd`: same as `YearRebuilt`.
- `BsmtFullBath`: because this is a binary feature (1 or 0), it's already technically one-hot encoded.
- `BsmtHalfBath`: same as `BsmtFullBath`.
- `GarageYrBlt`: this identifies the year the garage was built -- should be categorical.
- `MoSold`: this identifies the month the house was sold -- should be categorical.
- `YrSold`: same as MoSold, except with year instead of month.

In [5]:
# drop useless features
num_df.drop('Id', axis=1, inplace=True)

# drop categorical features
extracted_cat_features = ['MSSubClass', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 
                          'BsmtHalfBath', 'GarageYrBlt', 'MoSold', 'YrSold']
num_df.drop(extracted_cat_features, axis=1, inplace=True)

num_df.head()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,2,548,0,61,0,0,0,0,0,208500
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,2,460,298,0,0,0,0,0,0,181500
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,2,608,0,42,0,0,0,0,0,223500
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,3,642,0,35,272,0,0,0,0,140000
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,3,836,192,84,0,0,0,0,0,250000


Next, we'll identify any columns that contain `NaN` values. These values should be converted to 0.

In [6]:
# convert nan cells to 0
print(num_df.columns[num_df.isna().any()])
num_df.fillna(0, inplace=True)
print(num_df.columns[num_df.isna().any()])

Index(['LotFrontage', 'MasVnrArea'], dtype='object')
Index([], dtype='object')


Now, `num_df` should be ready for training.

### Processing categorical features
Some data is categorical, but an inherent order is implied; such as small, medium, and large. This type of feature engineering is called *ordinal encoding*.

In [7]:
# add dropped categorical features from the numerical df to the categorical df
cat_df = pd.concat([cat_df, df[extracted_cat_features]], axis=1)
cat_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,SaleType,SaleCondition,MSSubClass,YearBuilt,YearRemodAdd,BsmtFullBath,BsmtHalfBath,GarageYrBlt,MoSold,YrSold
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,WD,Normal,60,2003,2003,1,0,2003.0,2,2008
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,WD,Normal,20,1976,1976,0,1,1976.0,5,2007
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,WD,Normal,60,2001,2002,1,0,2001.0,9,2008
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,WD,Abnorml,70,1915,1970,1,0,1998.0,2,2006
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,WD,Normal,60,2000,2000,1,0,2000.0,12,2008


In [8]:
cat_df.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'MSSubClass', 'YearBuilt', 'YearRemodAdd',
       'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'MoSold', 'YrSold'],
      dtype='object')

Here, we identify the categories that are inherently ordinal, and should be converted.
- `LandSlope`
- `BsmtQual`
- `BsmtCond`
- `BsmtFinType1`
- `BsmtFinType2`
- `HeatingQC`
- `Electrical`
- `KitchenQual`
- `Functional`
- `FireplaceQu`
- `GarageQual`
- `GarageCond`
- `PoolQC`
One caveat is that for some of these like `GarageQual`, while the data is inherently ordinal, there are also values like `NaN` for homes without garages. So a home without a garage may not necessarily be worse than a home with a bad quality garage.

In [9]:
extracted_ord_features = ['LandSlope', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 
                          'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
                          'GarageQual', 'GarageCond', 'PoolQC']
cat_df.drop(extracted_ord_features, axis=1, inplace=True)
cat_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,Neighborhood,Condition1,Condition2,...,SaleType,SaleCondition,MSSubClass,YearBuilt,YearRemodAdd,BsmtFullBath,BsmtHalfBath,GarageYrBlt,MoSold,YrSold
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2003,2003,1,0,2003.0,2,2008
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Veenker,Feedr,Norm,...,WD,Normal,20,1976,1976,0,1,1976.0,5,2007
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2001,2002,1,0,2001.0,9,2008
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Crawfor,Norm,Norm,...,WD,Abnorml,70,1915,1970,1,0,1998.0,2,2006
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,NoRidge,Norm,Norm,...,WD,Normal,60,2000,2000,1,0,2000.0,12,2008


Now, we'll use one-hot encoding to encode these categorical features into numeric features.

In [10]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False)
enc.set_output(transform='pandas')
encoded_cat_df = enc.fit_transform(cat_df)
encoded_cat_df

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_nan,...,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1456,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1457,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1458,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Now, `cat_df` should be ready for training.

### Processing ordinal features
Now, all that's left is to convert the remaining categorical features into ordinal features.

In [11]:
extracted_ord_features

['LandSlope',
 'BsmtQual',
 'BsmtCond',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC']

In [12]:
ord_df = df[extracted_ord_features]
ord_df

,LandSlope,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC
0,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,NaN,TA,TA,NaN
1,Gtl,Gd,TA,ALQ,Unf,Ex,SBrkr,TA,Typ,TA,TA,TA,NaN
2,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,TA,TA,TA,NaN
3,Gtl,TA,Gd,ALQ,Unf,Gd,SBrkr,Gd,Typ,Gd,TA,TA,NaN
4,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,TA,TA,TA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Gtl,Gd,TA,Unf,Unf,Ex,SBrkr,TA,Typ,TA,TA,TA,NaN
1456,Gtl,Gd,TA,ALQ,Rec,TA,SBrkr,TA,Min1,TA,TA,TA,NaN
1457,Gtl,TA,Gd,GLQ,Unf,Ex,SBrkr,Gd,Typ,Gd,TA,TA,NaN
1458,Gtl,TA,TA,GLQ,Rec,Gd,FuseA,Gd,Typ,NaN,TA,TA,NaN


In [13]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(encoded_missing_value=-1)
enc.set_output(transform='pandas')
encoded_ord_df = enc.fit_transform(ord_df)
enc.categories_

[array(['Gtl', 'Mod', 'Sev'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'TA', nan], dtype=object),
 array(['Fa', 'Gd', 'Po', 'TA', nan], dtype=object),
 array(['ALQ', 'BLQ', 'GLQ', 'LwQ', 'Rec', 'Unf', nan], dtype=object),
 array(['ALQ', 'BLQ', 'GLQ', 'LwQ', 'Rec', 'Unf', nan], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA'], dtype=object),
 array(['FuseA', 'FuseF', 'FuseP', 'Mix', 'SBrkr', nan], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'TA'], dtype=object),
 array(['Maj1', 'Maj2', 'Min1', 'Min2', 'Mod', 'Sev', 'Typ'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA', nan], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA', nan], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA', nan], dtype=object),
 array(['Ex', 'Fa', 'Gd', nan], dtype=object)]

Here, the values are encoded such that values that are smaller are good, while values that are larger are bad. We've currently decided to encode `NaN` values as -1.

In [14]:
encoded_ord_df

,LandSlope,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC
0,0.0,2.0,3.0,2.0,5.0,0.0,4.0,2.0,6.0,-1.0,4.0,4.0,-1.0
1,0.0,2.0,3.0,0.0,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0
2,0.0,2.0,3.0,2.0,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0
3,0.0,3.0,1.0,0.0,5.0,2.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0
4,0.0,2.0,3.0,2.0,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,2.0,3.0,5.0,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0
1456,0.0,2.0,3.0,0.0,4.0,4.0,4.0,3.0,2.0,4.0,4.0,4.0,-1.0
1457,0.0,3.0,1.0,2.0,5.0,0.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0
1458,0.0,3.0,3.0,2.0,4.0,2.0,0.0,2.0,6.0,-1.0,4.0,4.0,-1.0


## Checkpoint -- finalizing the training set
We now have three dataframes -- `num_df`, `encoded_cat_df`, and `encoded_ord_df`. We'll now combine these three, moving the target column at the end.

In [15]:
print(num_df.shape, encoded_cat_df.shape, encoded_ord_df.shape)

(1460, 29) (1460, 506) (1460, 13)


In [16]:
target_col = num_df["SalePrice"]
num_df.drop("SalePrice", axis=1, inplace=True)
target_col

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [17]:
train_df = pd.concat([num_df, encoded_cat_df, encoded_ord_df, target_col], axis=1)
train_df

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC,SalePrice
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,5.0,0.0,4.0,2.0,6.0,-1.0,4.0,4.0,-1.0,208500
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0,181500
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0,223500
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,5.0,2.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0,140000
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,5,0.0,0,0,953,953,953,...,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0,175000
1456,85.0,13175,6,6,119.0,790,163,589,1542,2073,...,4.0,4.0,4.0,3.0,2.0,4.0,4.0,4.0,-1.0,210000
1457,66.0,9042,7,9,0.0,275,0,877,1152,1188,...,5.0,0.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0,266500
1458,68.0,9717,5,6,0.0,49,1029,0,1078,1078,...,4.0,2.0,0.0,2.0,6.0,-1.0,4.0,4.0,-1.0,142125


# Testing impact of NaN values
## Identifying columns with NaN values

In [18]:
from data_processor import DataProcessor
import pandas as pd

df = pd.read_csv('../data/train.csv')
dp = DataProcessor(df)

# create numerical dataframe
num_df = pd.concat([df[dp.numerical_cols].copy(), df['SalePrice']].copy(), axis=1)
num_df.drop('Id', axis=1, inplace=True)
display(num_df)

# check for NaNs
nan_counts = num_df.isna().sum()
print('Columns with NaN values')
display(nan_counts[nan_counts > 0])

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,2,548,0,61,0,0,0,0,0,208500
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,2,460,298,0,0,0,0,0,0,181500
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,2,608,0,42,0,0,0,0,0,223500
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,3,642,0,35,272,0,0,0,0,140000
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,3,836,192,84,0,0,0,0,0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,5,0.0,0,0,953,953,953,...,2,460,0,40,0,0,0,0,0,175000
1456,85.0,13175,6,6,119.0,790,163,589,1542,2073,...,2,500,349,0,0,0,0,0,0,210000
1457,66.0,9042,7,9,0.0,275,0,877,1152,1188,...,1,252,0,60,0,0,0,0,2500,266500
1458,68.0,9717,5,6,0.0,49,1029,0,1078,1078,...,1,240,366,0,112,0,0,0,0,142125


Columns with NaN values


LotFrontage    259
MasVnrArea       8
dtype: int64

In [19]:
# create ordinal dataframe
ord_df = df[dp.ordinal_cols].copy()
display(ord_df)

# check for NaNs
nan_counts = ord_df.isna().sum()
print('Columns with NaN values')
display(nan_counts[nan_counts > 0])

,LandSlope,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC
0,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,NaN,TA,TA,NaN
1,Gtl,Gd,TA,ALQ,Unf,Ex,SBrkr,TA,Typ,TA,TA,TA,NaN
2,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,TA,TA,TA,NaN
3,Gtl,TA,Gd,ALQ,Unf,Gd,SBrkr,Gd,Typ,Gd,TA,TA,NaN
4,Gtl,Gd,TA,GLQ,Unf,Ex,SBrkr,Gd,Typ,TA,TA,TA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Gtl,Gd,TA,Unf,Unf,Ex,SBrkr,TA,Typ,TA,TA,TA,NaN
1456,Gtl,Gd,TA,ALQ,Rec,TA,SBrkr,TA,Min1,TA,TA,TA,NaN
1457,Gtl,TA,Gd,GLQ,Unf,Ex,SBrkr,Gd,Typ,Gd,TA,TA,NaN
1458,Gtl,TA,TA,GLQ,Rec,Gd,FuseA,Gd,Typ,NaN,TA,TA,NaN


Columns with NaN values


BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageQual        81
GarageCond        81
PoolQC          1453
dtype: int64

In [20]:
# create categorical dataframe
cat_df = df[dp.categorical_cols].copy()
display(cat_df)

# check for NaNs
nan_counts = cat_df.isna().sum()
print('Columns with NaN values')
display(nan_counts[nan_counts > 0])

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,Neighborhood,Condition1,Condition2,...,SaleType,SaleCondition,MSSubClass,YearBuilt,YearRemodAdd,BsmtFullBath,BsmtHalfBath,GarageYrBlt,MoSold,YrSold
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2003,2003,1,0,2003.0,2,2008
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Veenker,Feedr,Norm,...,WD,Normal,20,1976,1976,0,1,1976.0,5,2007
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2001,2002,1,0,2001.0,9,2008
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Crawfor,Norm,Norm,...,WD,Abnorml,70,1915,1970,1,0,1998.0,2,2006
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,NoRidge,Norm,Norm,...,WD,Normal,60,2000,2000,1,0,2000.0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gilbert,Norm,Norm,...,WD,Normal,60,1999,2000,0,0,1999.0,8,2007
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,NWAmes,Norm,Norm,...,WD,Normal,20,1978,1988,1,0,1978.0,2,2010
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Crawfor,Norm,Norm,...,WD,Normal,70,1941,2006,0,0,1941.0,5,2010
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,NAmes,Norm,Norm,...,WD,Normal,20,1950,1996,1,0,1950.0,4,2010


Columns with NaN values


Alley           1369
MasVnrType       872
BsmtExposure      38
GarageType        81
GarageFinish      81
Fence           1179
MiscFeature     1406
GarageYrBlt       81
dtype: int64

In [21]:
# create data with no NaNs
num_df.dropna(axis=1, how='any', inplace=True)
ord_df.dropna(axis=1, how='any', inplace=True)

# encode ordinal data
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder().set_output(transform='pandas')
enc_ord_df = enc.fit_transform(ord_df)

no_nan_num_df = pd.concat([num_df, enc_ord_df], axis=1)
no_nan_num_df

,LotArea,OverallQual,OverallCond,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice,LandSlope,HeatingQC,KitchenQual,Functional
0,8450,7,5,706,0,150,856,856,854,0,...,0,0,0,0,0,208500,0.0,0.0,2.0,6.0
1,9600,6,8,978,0,284,1262,1262,0,0,...,0,0,0,0,0,181500,0.0,0.0,3.0,6.0
2,11250,7,5,486,0,434,920,920,866,0,...,0,0,0,0,0,223500,0.0,0.0,2.0,6.0
3,9550,7,5,216,0,540,756,961,756,0,...,272,0,0,0,0,140000,0.0,2.0,2.0,6.0
4,14260,8,5,655,0,490,1145,1145,1053,0,...,0,0,0,0,0,250000,0.0,0.0,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,5,0,0,953,953,953,694,0,...,0,0,0,0,0,175000,0.0,0.0,3.0,6.0
1456,13175,6,6,790,163,589,1542,2073,0,0,...,0,0,0,0,0,210000,0.0,4.0,3.0,2.0
1457,9042,7,9,275,0,877,1152,1188,1152,0,...,0,0,0,0,2500,266500,0.0,0.0,2.0,6.0
1458,9717,5,6,49,1029,0,1078,1078,0,0,...,112,0,0,0,0,142125,0.0,2.0,2.0,6.0


In [22]:
df = pd.read_csv('../data/train.csv')
dp = DataProcessor(df)

yes_nan_num_df = dp.numerical_data()
yes_nan_num_df

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC,SalePrice
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,5.0,0.0,4.0,2.0,6.0,-1.0,4.0,4.0,-1.0,208500
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0,181500
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0,223500
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,5.0,2.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0,140000
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,5,0.0,0,0,953,953,953,...,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0,175000
1456,85.0,13175,6,6,119.0,790,163,589,1542,2073,...,4.0,4.0,4.0,3.0,2.0,4.0,4.0,4.0,-1.0,210000
1457,66.0,9042,7,9,0.0,275,0,877,1152,1188,...,5.0,0.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0,266500
1458,68.0,9717,5,6,0.0,49,1029,0,1078,1078,...,4.0,2.0,0.0,2.0,6.0,-1.0,4.0,4.0,-1.0,142125


## Testing
### Numerical data, imputed vs numerical data, removed NaNs

In [23]:
# proper training pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# data; X and y
y = no_nan_num_df['SalePrice']
no_nan_num_df.drop('SalePrice', axis=1, inplace=True)
X = no_nan_num_df

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 34445, normalized rmse: 19.0%'

In [24]:
# data; X and y
y = yes_nan_num_df['SalePrice']
yes_nan_num_df.drop('SalePrice', axis=1, inplace=True)
X = yes_nan_num_df

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 35135, normalized rmse: 19.4%'

### Whole set imputed vs whole set NaN removed

In [25]:
cat_df.dropna(axis=1, how='any', inplace=True)
num_df.shape, enc_ord_df.shape, cat_df.shape

((1460, 27), (1460, 4), (1460, 30))

In [26]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
enc_cat_df = ohe.fit_transform(cat_df)
enc_cat_df

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1456,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1457,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1458,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
no_imputed_data = pd.concat([num_df, enc_ord_df, enc_cat_df], axis=1)
no_imputed_data

,LotArea,OverallQual,OverallCond,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,8450,7,5,706,0,150,856,856,854,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,9600,6,8,978,0,284,1262,1262,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,11250,7,5,486,0,434,920,920,866,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9550,7,5,216,0,540,756,961,756,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,14260,8,5,655,0,490,1145,1145,1053,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,5,0,0,953,953,953,694,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1456,13175,6,6,790,163,589,1542,2073,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1457,9042,7,9,275,0,877,1152,1188,1152,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1458,9717,5,6,49,1029,0,1078,1078,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [28]:
# data; X and y
y = no_imputed_data['SalePrice']
no_imputed_data.drop('SalePrice', axis=1, inplace=True)
X = no_imputed_data

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 178424267935604192, normalized rmse: 98619880914163.8%'

In [29]:
# with pca
from sklearn.decomposition import PCA

# pipeline
regr = make_pipeline(StandardScaler(), PCA(n_components=0.95), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 34859, normalized rmse: 19.3%'

In [30]:
yes_imputed_data = dp.complete_data()

# data; X and y
y = yes_imputed_data['SalePrice']
yes_imputed_data.drop('SalePrice', axis=1, inplace=True)
X = yes_imputed_data

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 1741287657183436, normalized rmse: 962456415686.2%'

In [31]:
# with pca

# pipeline
regr = make_pipeline(StandardScaler(), PCA(n_components=0.95), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 33690, normalized rmse: 18.6%'

## Results: Removed vs Imputed
1. **Numerical**: With the numerical dataset (numerical + ordinal), NaN-removed performed 0.4% *better* than imputed.
2. **Whole set**: With the whole dataset (numerical + ordinal + one hot encoded categories), NaN-removed performed *better* than imputed.
3. **Whole set + PCA**: With the whole dataset with PCA performed, there is a 0.7% *decline* in NRMSE performance from the NaN-removed dataset to the imputed dataset.

# Testing the inclusion of years as numerical vs categorical
The goal is to determine if using years as numerical data is more or less performant compared to using years as one hot encoded data.

## Testing

In [32]:
from data_processor import DataProcessor
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# load numerical data
df = pd.read_csv('../data/train.csv')
dp = DataProcessor(df)
num_df = dp.numerical_data()
num_df.drop('SalePrice', axis=1, inplace=True)

# create a numerical dataset with years -- categorical and numerical
year_num_df = df['YrSold']

ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
year_cat_df = ohe.fit_transform(pd.DataFrame(df['YrSold']))

display(year_num_df.head(), year_cat_df.head())

0    2008
1    2007
2    2008
3    2006
4    2008
Name: YrSold, dtype: int64

,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0


### Baseline -- numerical features, no years

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = num_df
y = df['SalePrice']

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 35135, normalized rmse: 19.4%'

### Numerical features with numerical years

In [34]:
# data; X and y
X = pd.concat([num_df, year_num_df], axis=1)

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 35146, normalized rmse: 19.4%'

### Numerical features with categorical years

In [35]:
X = pd.concat([num_df, year_cat_df], axis=1)

# pipeline
regr = make_pipeline(StandardScaler(), LinearRegression())
neg_rmse_scores = cross_val_score(regr, X, y, cv=10, scoring='neg_root_mean_squared_error')

# calc metrics
rmse = -neg_rmse_scores.mean()
nrmse = rmse / y.mean()
percent_nrmse = f'{round(nrmse * 100, 1)}%'

f'rmse: {round(rmse)}, normalized rmse: {percent_nrmse}'

'rmse: 35154, normalized rmse: 19.4%'

## Results
The baseline with numerical features and no year had the best rmse, followed by numerical years, followed by categorical years.

That is, we could say that not including the years would be best; if we were to, including them as numerical would be better than as categorical.

However, gains/losses were so minimal, normalized rmse didn't even move by 0.1%. Since the change is so imperceptible, it likely does not matter, and is not worth refactoring the current implementation of `DataProcessor`.